In [1]:
import pandas as pd
import numpy as np
import bcolz as bz
import torch
from torch import nn

In [2]:
from fastai.vision import *

In [3]:
SIZE = 456
SITE = 3 # Site: 1:site1, 2:site2, 3:site1 and 2

LR = 1e-5
BS = 64
EPOCHS = 100

In [4]:
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader

In [5]:
train_bc = bz.open("/data/rcic/actv_train")

In [6]:
arr = train_bc[[2,5,7,8]]

In [7]:
train_bc.shape

(73024, 2048)

In [8]:
train_bc.flush()

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    

SEED = 0
seed_everything(SEED)


In [10]:
from pathlib import Path
print("Loading training dataframe")
DATA = Path("/mnt/disk4/cell/")

train_df = pd.read_csv(DATA/'train.csv')
train_df.head(10)

Loading training dataframe


,id_code,experiment,plate,well,sirna
0,HEPG2-01_1_B03,HEPG2-01,1,B03,513
1,HEPG2-01_1_B04,HEPG2-01,1,B04,840
2,HEPG2-01_1_B05,HEPG2-01,1,B05,1020
3,HEPG2-01_1_B06,HEPG2-01,1,B06,254
4,HEPG2-01_1_B07,HEPG2-01,1,B07,144
5,HEPG2-01_1_B08,HEPG2-01,1,B08,503
6,HEPG2-01_1_B09,HEPG2-01,1,B09,188
7,HEPG2-01_1_B10,HEPG2-01,1,B10,700
8,HEPG2-01_1_B11,HEPG2-01,1,B11,1100
9,HEPG2-01_1_B12,HEPG2-01,1,B12,611


In [11]:
groups = np.load("groups.npy")

In [12]:
def generate_df(train_df,sample_num=1):
    train_df['path'] = train_df['experiment'].str.cat(train_df['plate'].astype(str).str.cat(train_df['well'],sep='/'),sep='/Plate') + '_s'+str(sample_num) + '_w'
    train_df["pname"] = train_df.apply(lambda x:x["experiment"]+"-"+x["plate"].__str__(), axis=1)
#     train_df["grp"] = train_df.sirna.apply(lambda x:groups[x])
    train_df = train_df.drop(columns=['id_code','experiment','plate','well']).reindex(columns=['path','sirna',"pname"])
    
    return train_df

In [13]:
site1_train_df = generate_df(train_df)  
site2_train_df = generate_df(train_df, sample_num=2)

if SITE==1: # only site1
    proc_train_df = site1_train_df 
elif SITE==2 : # only site2
    proc_train_df = site2_train_df
elif SITE==3 :
    proc_train_df = pd.concat([site1_train_df,site2_train_df],axis=0 ).reset_index().drop("index",axis=1).head(73024)
#     proc_train_df.to_csv("train_with_bc.csv")

In [29]:
act_df = proc_train_df.reset_index().rename(columns = {"index":"actid"})

In [30]:
act_df["grp"]=act_df["sirna"].apply(lambda x:groups[x])

In [32]:
split_ = (np.random.rand(len(act_df))<0.965)

In [33]:
train_act_df = act_df[split_]
val_act_df = act_df[~split_]
len(train_act_df),len(val_act_df)

(70390, 2634)

In [34]:
import math
class actDs(Dataset):
    def __init__(self,df,ba,bs=BS):
        self.df = df.sample(frac = 1.).reset_index().drop("index",axis=1)
        self.ba = ba
        self.bs = bs
        
    def __len__(self):
        return math.ceil(float(len(self.df))/float(self.bs))
    
    def __getitem__(self,idx):
        df_ = self.df[idx*self.bs:(idx+1)*self.bs]
        act_arr = self.ba[list(df_.actid)]
        self.ba.flush()
        return act_arr,df_.sirna.values, df_.grp.values

In [35]:
train_ds = actDs(train_act_df,train_bc)
val_ds = actDs(val_act_df,train_bc)

In [18]:
train_dl = DataLoader(train_ds,batch_size=1, num_workers=4,)

In [36]:
from forgebox.ftorch.train import Trainer
from forgebox.ftorch.callbacks import stat

In [37]:
class aLearner(nn.Module):
    def __init__(self,):
        super().__init__()
        self.top = nn.Sequential(*[
            nn.Linear(2048,2048, bias=False),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Linear(2048,1108, bias=False),
            nn.BatchNorm1d(1108),
            nn.Softmax(),])
        
    def forward(self,x):
        return self.top(x)        

In [38]:
CUDA = torch.cuda.is_available()

In [39]:
model = aLearner().train()
if CUDA:
    model.cuda()

In [40]:
from forgebox.ftorch.metrics import accuracy

In [41]:
opt = torch.optim.Adam(model.parameters(), lr=1e-4)

In [42]:
crit = nn.CrossEntropyLoss()

In [43]:
t = Trainer(train_ds, val_dataset=val_ds,
            batch_size=1, 
            num_workers=4, 
            callbacks=[stat], val_callbacks=[stat])


In [44]:
@t.step_train
def action(batch):
    if batch.i ==0:
        model.train()
    opt.zero_grad()
    x,y1,y2 = batch.data
    x  =x[0].float(); y1 = y1[0]
    if CUDA:
        x = x.cuda()
        y1 = y1.cuda()
        y2 = y2.cuda()
        
    y1_ = model(x)
    loss = crit(y1_,y1)
    acc = accuracy(y1_,y1)
    loss.backward()
    opt.step()
    return {"loss":loss.item(),"acc":acc.item()}

@t.step_val
def val_action(batch):
    if batch.i ==0:
        model.eval()
#     opt.zero_grad()
    x,y1,y2 = batch.data
    x  =x[0].float(); y1 = y1[0]
    if CUDA:
        x = x.cuda()
        y1 = y1.cuda()
        y2 = y2.cuda()
        
    y1_ = model(x)
    loss = crit(y1_,y1)
    acc = accuracy(y1_,y1)
#     loss.backward()
#     opt.step()
    return {"loss":loss.item(),"acc":acc.item()}

In [45]:
t.train(50)

/opt/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


,loss,acc,epoch,iter,timestamp
mean,6.949199,0.220131,0.0,549.5,0.018473
min,6.856333,0.000000,0.0,0.0,0.000000
max,7.010438,0.531250,0.0,1099.0,20.320372


,loss,acc,epoch,iter,timestamp
mean,6.732979,0.411979,0.0,20.5,0.017826
min,6.592898,0.100000,0.0,0.0,0.000000
max,6.996486,0.593750,0.0,41.0,0.748686


,loss,acc,epoch,iter,timestamp
mean,6.822542,0.511285,1.0,549.5,0.016149
min,6.730528,0.171875,1.0,0.0,0.000000
max,6.982502,0.703125,1.0,1099.0,17.763589


,loss,acc,epoch,iter,timestamp
mean,6.601595,0.552976,1.0,20.5,0.018113
min,6.458587,0.100000,1.0,0.0,0.000000
max,6.986842,0.750000,1.0,41.0,0.760735


...

,loss,acc,epoch,iter,timestamp
mean,6.053308,0.971333,43.0,549.5,0.017077
min,6.017522,0.875000,43.0,0.0,0.000000
max,6.165321,1.000000,43.0,1099.0,18.784872


,loss,acc,epoch,iter,timestamp
mean,6.317550,0.807589,43.0,20.5,0.021840
min,6.205634,0.700000,43.0,0.0,0.000000
max,6.577052,0.906250,43.0,41.0,0.917277


,loss,acc,epoch,iter,timestamp
mean,6.051644,0.972512,44.0,549.5,0.019045
min,6.017340,0.890625,44.0,0.0,0.000000
max,6.153747,1.000000,44.0,1099.0,20.949803


Exception in thread Thread-186:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.6/site-packages/torch/utils/data/_utils/pin_memory.py", line 21, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/opt/anaconda3/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/opt/anaconda3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 284, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/opt/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authke

KeyboardInterrupt: 

In [91]:
test_df = pd.read_csv(DATA/'test.csv')
test_df.head(10)

,id_code,experiment,plate,well
0,HEPG2-08_1_B03,HEPG2-08,1,B03
1,HEPG2-08_1_B04,HEPG2-08,1,B04
2,HEPG2-08_1_B05,HEPG2-08,1,B05
3,HEPG2-08_1_B06,HEPG2-08,1,B06
4,HEPG2-08_1_B07,HEPG2-08,1,B07
5,HEPG2-08_1_B08,HEPG2-08,1,B08
6,HEPG2-08_1_B09,HEPG2-08,1,B09
7,HEPG2-08_1_B10,HEPG2-08,1,B10
8,HEPG2-08_1_B11,HEPG2-08,1,B11
9,HEPG2-08_1_B12,HEPG2-08,1,B12


In [92]:
test1_bc = bz.open("/data/rcic/actv_test1")

In [93]:
test1_bc.shape

(19897, 2048)

In [94]:
test1_df = generate_df(test_df, sample_num=1).reset_index().rename(columns={"index":"actid"})

In [95]:
test1_df

,actid,path,sirna,pname
0,0,HEPG2-08/Plate1/B03_s1_w,NaN,HEPG2-08-1
1,1,HEPG2-08/Plate1/B04_s1_w,NaN,HEPG2-08-1
2,2,HEPG2-08/Plate1/B05_s1_w,NaN,HEPG2-08-1
3,3,HEPG2-08/Plate1/B06_s1_w,NaN,HEPG2-08-1
4,4,HEPG2-08/Plate1/B07_s1_w,NaN,HEPG2-08-1
...,...,...,...,...
19892,19892,U2OS-05/Plate4/O19_s1_w,NaN,U2OS-05-4
19893,19893,U2OS-05/Plate4/O20_s1_w,NaN,U2OS-05-4
19894,19894,U2OS-05/Plate4/O21_s1_w,NaN,U2OS-05-4
19895,19895,U2OS-05/Plate4/O22_s1_w,NaN,U2OS-05-4


In [96]:
force_group = np.load("force_group.npy", allow_pickle=True).tolist()

In [97]:
test1_df["grp"] = test1_df.pname.apply(lambda x:force_group[x])

In [110]:
test1_ds = actDs(test1_df,test_bc)
t1 = Trainer(test1_ds,shuffle=False, batch_size=1)

In [111]:
allpred1 = []

In [112]:
@t1.step_train
def action(batch):
    model.eval()
    if batch.i ==0:
        model.eval()
#     opt.zero_grad()
    x,y1,y2 = batch.data
    x  =x[0].float(); y1 = y1[0]
    if CUDA:
        x = x.cuda()
        y1 = y1.cuda()
        y2 = y2.cuda()
        
    y1_ = model(x)
    allpred1.append(y1_.cpu().data.numpy())
    return {"maxval":y1_.argmax(dim=-1)[0].item()}

In [113]:
t1.train(1)

/opt/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [120]:
np.save("shortpred1.npy",np.concatenate(allpred1, axis=0))